<a href="https://colab.research.google.com/github/EmiLaPiola/NumLabIMC/blob/main/Copia_de_TP_autom%C3%A1tico.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [29]:
from google.colab import drive
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, average_precision_score, roc_auc_score
from IPython.display import display, HTML

df = pd.read_csv('data.csv')

# Ejercicio 1 — Separación de datos

> *Evaluar y justificar cómo separarán sus datos para desarrollo y para evaluación. No usar `train_test_split` de sklearn.*

Antes de separar los 500 datos en conjuntos de desarrollo y test, analizamos la variable `target` para contar cuántas instancias son positivas y cuántas negativas. La idea es darnos cuenta si las clases estan balanceadas o no . En el segundo caso , deberiamos hacer una división estratificada.


In [7]:
df['target'].value_counts()

,count
target,
0,353
1,147


El data set esta muy desbalanceado, hay mucha mas instancias con mal pronostico ( target 0 ) y solo un 29.4% de los datos tienen buen pronostico (target 1). Así que para separar nuestros datos no lo haremos al azar. La separacion sera estratificada para que la proporicon de la clase minoritaria ( buen pronostico) se preserve en ambos conjuntos. Utilizaremos 80 % de los datos para train y 20 % para control.

Haciendo la partición en 80-20, tenemos 400 instancias en el conjunto de desarrollo y 100 en el de control. Para garantizar la estratificación, seleccionamos aproximadamente el 20% de los casos positivos y el 20% de los negativos para formar el conjunto de control. De esta manera , ambos conjuntos mantienen la proporción original de clases.

Inicialmente entonces creamos dos data sets distintos. Uno destinado para el desarrolo y otro para la evaluación.

In [10]:

# separamos positivos (buen pronostico) y negativos (mal pronsotico)
positivos = df[df['target'] == True]
negativos = df[df['target'] == False]

# seleccionamos al azar el 20% para control
control_positivos = positivos.sample(frac=0.2, random_state=42)
control_negativos = negativos.sample(frac=0.2, random_state=42)

# concatenamos el set de control
NO_TOCAR_set = pd.concat([control_positivos, control_negativos])   # NO TOCAR

# El resto de los datos queda para desarrollo
desarrollo_set = df.drop(NO_TOCAR_set.index)

print(f"Desarrollo: {len(desarrollo_set)} instancias")
print(f"Control: {len(NO_TOCAR_set)} instancias")

Desarrollo: 400 instancias
Control: 100 instancias


# Ejercicio 2 — Entrenamiento

> a) *Entrenar un árbol de decisión con altura máxima 3 y el resto de los hiperparámetros en default. Estimar la performance del modelo utilizando K-fold cross validation con K=5, con las métricas Accuracy, Area Under the Precision-Recall Curve (AUPRC), y Area Under the Receiver Operating Characteristic Curve (AUCROC).*

Para hacer CV con 5 folds, primero vamos a hacer una division estratificada de estos . Como ya dijimos antes, tenemos muy pocos positivos y muchos negativos, las clases estan muy desbalanceadas, por ende, si hicieramos la division de Kfold-CV al azar, podríamos obtener un modelo entrenado con una sola clase y métricas engañosas ... no es la idea.
Asi que vamos a usar K-fold-CV estratificado para mantener la proporcion de clases en cada fold.

In [31]:
# hacemos 5 folds estratificados para que haya las mismas proprciones de clases minoritarias y mayoritarias en todos los folds.

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state = 22)

# separamos de nuestro data set la columna "target"

desarrollo_set = desarrollo_set.reset_index(drop=True)
x_desarrollo = desarrollo_set.drop('target', axis=1)
y_desarrollo = desarrollo_set['target']


# inicializamos nuestras listas para guadar las metricas

vector_accuracy_train= []       # accuracy en training en cada fold
vector_accuracy_validacion= []  # lo mismo en validacion

vector_auprc_train  = []        # AUPRC en training en cada fold
vector_auprc_validacion = []    # lo mismo pero en val

vector_auroc_train = []         # AUC ROC en training en cada fold
vector_auroc_validacion=[]      # lo mismo pero en val


# creamos un array para guardar las predicciones
y_pred = np.empty(y_desarrollo.shape)
y_pred.fill(np.nan)

# lo mismo pero para guardar la probabilidad predicha de pertenecer a la clase positiva
# esto lo vamos a usar para calcular AUC-ROC y AUPRC
y_pred_prob = np.empty(y_desarrollo.shape)
y_pred_prob.fill(np.nan)


In [32]:
# generamos para cada fold una predicción
for train_index, test_index in folds.split(x_desarrollo,y_desarrollo):

        #saco el fold que no uso para entrenar
        kf_X_train, kf_X_test = x_desarrollo.iloc[train_index], x_desarrollo.iloc[test_index]
        kf_y_train, kf_y_test = y_desarrollo.iloc[train_index], y_desarrollo.iloc[test_index]

        # arbol de altura 3 con los datos de train
        arbol = DecisionTreeClassifier(max_depth=3)
        arbol.fit(kf_X_train, kf_y_train)
        y_pred_prob[test_index] = arbol.predict_proba(kf_X_test)[:, 1] # la prob pred de clase positiva para val de ese fold

        # hacemos las predicciones
        predictions = arbol.predict(kf_X_test)
        y_pred[test_index] = predictions

        # accuracy
        vector_accuracy_validacion.append(accuracy_score(kf_y_test, predictions))
        vector_accuracy_train.append(accuracy_score(kf_y_train, arbol.predict(kf_X_train)))


        # AUPRC
        auprc = average_precision_score(kf_y_test, arbol.predict_proba(kf_X_test)[:, 1])
        vector_auprc_validacion.append(auprc)
        vector_auprc_train.append(average_precision_score(kf_y_train, arbol.predict_proba(kf_X_train)[:, 1]))

        # AUROC
        auc_roc = roc_auc_score(kf_y_test, arbol.predict_proba(kf_X_test)[:, 1])
        vector_auroc_validacion.append(auc_roc)
        vector_auroc_train.append(roc_auc_score(kf_y_train, arbol.predict_proba(kf_X_train)[:, 1]))



In [33]:
# accuracy
print("Promedio de accuracys por fold:", round(np.mean(vector_accuracy_validacion), 4))
print("Accuracy global:", round(accuracy_score(y_desarrollo, y_pred), 4))
print()

# AUPRC
print("Promedio de AUPRC por fold:", round(np.mean(vector_auprc_validacion), 4))
print("AUPRC global:", round(average_precision_score(y_desarrollo, y_pred_prob), 4))
print()

# AUROC
print("Promedio de AUROC por fold:", round(np.mean(vector_auroc_validacion), 4))
print("AUROC global:", round(roc_auc_score(y_desarrollo, y_pred_prob), 4))

Promedio de accuracys por fold: 0.7125
Accuracy global: 0.7125

Promedio de AUPRC por fold: 0.4487
AUPRC global: 0.4369

Promedio de AUROC por fold: 0.6822
AUROC global: 0.6897


In [34]:
from IPython.display import Markdown, display

tabla_md = """
| Métrica  | Promedio Fold | Global |
|:--------:|:-------------:|:------:|
| Accuracy |     0.7075    | 0.705  |
| AUPRC    |     0.4359    | 0.4178 |
| AUROC    |     0.6732    | 0.675  |
"""

display(Markdown(tabla_md))



| Métrica  | Promedio Fold | Global |
|:--------:|:-------------:|:------:|
| Accuracy |     0.7075    | 0.705  |
| AUPRC    |     0.4359    | 0.4178 |
| AUROC    |     0.6732    | 0.675  |


A pesar de que obtuvimos un accuracy del 70%, este valor no es significativo debido al fuerte desbalance de clases del dataset. Ya de por si un clasificador trivial que predice siempre la clase negativa (mal pronóstico) ya alcanza esa predicción.

Si miramos los resultados del AUPRC, considerando que nosotras tenemos clase positiva minoritaria, un modelo trivial tendria aproximadamente un AUPRC ≈ 0.29. El nuestro nos dio mejor que eso, asi que en este caso al menos superamos al azar.

Un modelo totalmente random tendria un AUROC de 0.5. El nuestro es un poco mejor, pero igualmente claramente necesita mejoras.


En lo siguiente, completamos la tabla que se nos pide en el TP con los resultados obtenidos.

In [39]:
# creamos la tabla que se nos pide con los resultados obtenidos

tabla_resultados = pd.DataFrame({
    "Permutación": list(range(1, len(vector_accuracy_validacion)+1)),
    "Accuracy (training)": [round(x,3) for x in vector_accuracy_train],
    "Accuracy (validación)": [round(x,3) for x in vector_accuracy_validacion],
    "AUPRC (training)": [round(x,3) for x in vector_auprc_train],
    "AUPRC (validación)": [round(x,3) for x in vector_auprc_validacion],
    "AUC ROC (training)": [round(x,3) for x in vector_auroc_train],
    "AUC ROC (validación)": [round(x,3) for x in vector_auroc_validacion]
})



promedios = {
    "Permutación": "Promedio",
    "Accuracy (training)": round(np.mean(vector_accuracy_train), 3),
    "Accuracy (validación)": round(np.mean(vector_accuracy_validacion), 3),
    "AUPRC (training)": round(np.mean(vector_auprc_train), 3),
    "AUPRC (validación)": round(np.mean(vector_auprc_validacion), 3),
    "AUC ROC (training)": round(np.mean(vector_auroc_train), 3),
    "AUC ROC (validación)": round(np.mean(vector_auroc_validacion), 3)
}


globales = {
    "Permutación": "Global",
    "Accuracy (training)": "(NO)",
    "Accuracy (validación)": round(accuracy_score(y_desarrollo, y_pred), 3),
    "AUPRC (training)": "(NO)",
    "AUPRC (validación)": round(average_precision_score(y_desarrollo, y_pred_prob), 3),
    "AUC ROC (training)": "(NO)",
    "AUC ROC (validación)": round(roc_auc_score(y_desarrollo, y_pred_prob), 3)
}

tabla_resultados = pd.concat([
    tabla_resultados,
    pd.DataFrame([promedios]),
    pd.DataFrame([globales])
], ignore_index=True)

display(HTML(tabla_resultados.to_html(index=False, justify='center')))


Permutación,Accuracy (training),Accuracy (validación),AUPRC (training),AUPRC (validación),AUC ROC (training),AUC ROC (validación)
1,0.856,0.650,0.72,0.406,0.874,0.616
2,0.853,0.750,0.713,0.516,0.842,0.686
3,0.853,0.725,0.704,0.444,0.829,0.693
4,0.847,0.688,0.749,0.363,0.855,0.637
5,0.844,0.750,0.745,0.514,0.848,0.778
Promedio,0.851,0.712,0.726,0.449,0.849,0.682
Global,(NO),0.713,(NO),0.437,(NO),0.690


La clase positiva es la clase minoritaria . La AUPRC (Area Under Precision-Recall Curve) es muy sensible al desbalance de clases. Y como tenemos muy pocos positivos tiene sentido que nos de un promedio global muy bajo en la validacion de aupcr  # anotar cositas de las diapos de clase sobre estos temas xd .
Tambien tiene todo el sentido del mundo que eln los folds de entrenamiento nos de mucho mejor que en los sets de validacion ( trivial ) . Tanto roc como aupcr son sensibles a data set debalanceados como vimos en clase .

Tiene sentido que el promedio de igual al global en este caso particular porque los folds tienen todos el mismo tamaño... desarrollar :)

In [48]:
## como la clase mayoritaria son los negativos invertimos las etiquetas
## pruebitas extras cambiando clases xd
##  cambiando las etiquetas de las predicciones observamos que las metricas son mejroes


# ---- Datos ----
desarrollo_set = desarrollo_set.reset_index(drop=True)
x_desarrollo = desarrollo_set.drop('target', axis=1)
y_desarrollo = desarrollo_set['target']

# Invertimos la clase: ahora la clase mayoritaria es la "positiva"
y_desarrollo_invertida = 1 - y_desarrollo

# ---- Stratified K-Fold ----
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=22)

# ---- Vectores de métricas ----
vector_accuracy_train = []
vector_accuracy_validacion = []
vector_auprc_train = []
vector_auprc_validacion = []
vector_auroc_train = []
vector_auroc_validacion = []

# ---- Predicciones globales ----
y_pred = np.empty(y_desarrollo.shape)
y_pred.fill(np.nan)
y_pred_prob = np.empty(y_desarrollo.shape)
y_pred_prob.fill(np.nan)

# ---- Entrenamiento y evaluación por fold ----
for train_index, test_index in folds.split(x_desarrollo, y_desarrollo_invertida):
    kf_X_train, kf_X_test = x_desarrollo.iloc[train_index], x_desarrollo.iloc[test_index]
    kf_y_train, kf_y_test = y_desarrollo_invertida.iloc[train_index], y_desarrollo_invertida.iloc[test_index]

    arbol = DecisionTreeClassifier(max_depth=3)
    arbol.fit(kf_X_train, kf_y_train)

    y_pred_fold = arbol.predict(kf_X_test)
    y_pred_prob_fold = arbol.predict_proba(kf_X_test)[:, 1]

    y_pred[test_index] = y_pred_fold
    y_pred_prob[test_index] = y_pred_prob_fold

    vector_accuracy_train.append(accuracy_score(kf_y_train, arbol.predict(kf_X_train)))
    vector_accuracy_validacion.append(accuracy_score(kf_y_test, y_pred_fold))
    vector_auprc_train.append(average_precision_score(kf_y_train, arbol.predict_proba(kf_X_train)[:, 1]))
    vector_auprc_validacion.append(average_precision_score(kf_y_test, y_pred_prob_fold))
    vector_auroc_train.append(roc_auc_score(kf_y_train, arbol.predict_proba(kf_X_train)[:, 1]))
    vector_auroc_validacion.append(roc_auc_score(kf_y_test, y_pred_prob_fold))

# ---- Cálculo de métricas globales ----
accuracy_global = accuracy_score(y_desarrollo_invertida, y_pred)
auprc_global = average_precision_score(y_desarrollo_invertida, y_pred_prob)
auroc_global = roc_auc_score(y_desarrollo_invertida, y_pred_prob)

# ---- Tabla de resultados ----
tabla_resultados = pd.DataFrame({
    "Permutación": list(range(1, 6)),
    "Accuracy (training)": [round(x, 3) for x in vector_accuracy_train],
    "Accuracy (validación)": [round(x, 3) for x in vector_accuracy_validacion],
    "AUPRC (training)": [round(x, 3) for x in vector_auprc_train],
    "AUPRC (validación)": [round(x, 3) for x in vector_auprc_validacion],
    "AUC ROC (training)": [round(x, 3) for x in vector_auroc_train],
    "AUC ROC (validación)": [round(x, 3) for x in vector_auroc_validacion]
})

# Fila de promedios
promedios = {
    "Permutación": "Promedio",
    "Accuracy (training)": round(np.mean(vector_accuracy_train), 3),
    "Accuracy (validación)": round(np.mean(vector_accuracy_validacion), 3),
    "AUPRC (training)": round(np.mean(vector_auprc_train), 3),
    "AUPRC (validación)": round(np.mean(vector_auprc_validacion), 3),
    "AUC ROC (training)": round(np.mean(vector_auroc_train), 3),
    "AUC ROC (validación)": round(np.mean(vector_auroc_validacion), 3)
}

# Fila de globales (correctamente usando etiquetas invertidas)
globales = {
    "Permutación": "Global",
    "Accuracy (training)": "(NO)",
    "Accuracy (validación)": round(accuracy_global, 3),
    "AUPRC (training)": "(NO)",
    "AUPRC (validación)": round(auprc_global, 3),
    "AUC ROC (training)": "(NO)",
    "AUC ROC (validación)": round(auroc_global, 3)
}

# Unir todo
tabla_resultados = pd.concat([
    tabla_resultados,
    pd.DataFrame([promedios]),
    pd.DataFrame([globales])
], ignore_index=True)

# Mostrar tabla centrada
display(HTML(tabla_resultados.to_html(index=False, justify='center')))

Permutación,Accuracy (training),Accuracy (validación),AUPRC (training),AUPRC (validación),AUC ROC (training),AUC ROC (validación)
1,0.856,0.625,0.917,0.752,0.874,0.573
2,0.853,0.750,0.891,0.796,0.842,0.686
3,0.853,0.725,0.881,0.817,0.829,0.693
4,0.847,0.688,0.899,0.802,0.855,0.637
5,0.844,0.750,0.892,0.877,0.848,0.778
Promedio,0.851,0.708,0.896,0.809,0.849,0.674
Global,(NO),0.708,(NO),0.805,(NO),0.679


1.3) *Explorar las siguientes combinaciones de parámetros para árboles de decisión utilizando `ParameterGrid` de scikit learn. No está permitido utilizar `GridSearchCV` en este ejercicio.*

In [6]:
# Probamos disitnos parametros para nuestro arbol de decision

folds = StratifiedKFold(n_splits=5, shuffle=True, random_state = 22)

alturas = [1,2,3,5,10,None]
criterios = ['gini','entropy']
accuracy_validacion = {}
accuracy_train = {}

for a in alturas:
    for c in criterios:
        vector_accuracy_validacion= []  # aca vamos a guardar los resultados de acurracy para cada fold de la validacion
        vector_accuracy_train= [] # y del training

        # generamos para cada fold una predicción
        for train_index, test_index in folds.split(x_desarrollo,y_desarrollo):

          #saco el fold que no uso para entrenar
          kf_X_train, kf_X_test = x_desarrollo.iloc[train_index], x_desarrollo.iloc[test_index]
          kf_y_train, kf_y_test = y_desarrollo.iloc[train_index], y_desarrollo.iloc[test_index]

          # arbol fit con los datos de train
          arbol = DecisionTreeClassifier(max_depth=a, criterion=c)
          arbol.fit(kf_X_train, kf_y_train)

          y_pred_val = arbol.predict(kf_X_test)
          acc_val = accuracy_score(kf_y_test, y_pred_val)
          vector_accuracy_validacion.append(acc_val)

          y_pred_train = arbol.predict(kf_X_train)
          acc_train = accuracy_score(kf_y_train, y_pred_train)
          vector_accuracy_train.append(acc_train)

        # Guardamos el promedio de accuracy de los folds
        accuracy_validacion[(a, c)] = np.mean(vector_accuracy_validacion)
        accuracy_train[(a, c)] = np.mean(vector_accuracy_train)

print(accuracy_validacion)
print(accuracy_train)
# El mejor es gini con altura 5

import pandas as pd

# Lista para guardar filas de la tabla
tabla_resultados = []

# Recorremos las claves del diccionario (pares de (altura, criterio))
for clave in accuracy_train.keys():
    altura, criterio = clave
    acc_train = accuracy_train[clave]
    acc_val = accuracy_validacion[clave]

    # Agregamos una fila a la tabla
    tabla_resultados.append({
        'Altura': altura,
        'Criterio': criterio,
        'Accuracy Train': acc_train,
        'Accuracy Validación': acc_val
    })

# Creamos el DataFrame
df_resultados = pd.DataFrame(tabla_resultados)

# Mostramos la tabla ordenada por Altura y Criterio
df_resultados = df_resultados.sort_values(by=['Altura', 'Criterio']).reset_index(drop=True)
print(df_resultados)


{(1, 'gini'): np.float64(0.6775), (1, 'entropy'): np.float64(0.6775), (2, 'gini'): np.float64(0.7224999999999999), (2, 'entropy'): np.float64(0.735), (3, 'gini'): np.float64(0.71), (3, 'entropy'): np.float64(0.7150000000000001), (5, 'gini'): np.float64(0.71), (5, 'entropy'): np.float64(0.6799999999999999), (10, 'gini'): np.float64(0.6925000000000001), (10, 'entropy'): np.float64(0.6575), (None, 'gini'): np.float64(0.6925), (None, 'entropy'): np.float64(0.6475)}
{(1, 'gini'): np.float64(0.7093750000000001), (1, 'entropy'): np.float64(0.7093750000000001), (2, 'gini'): np.float64(0.784375), (2, 'entropy'): np.float64(0.779375), (3, 'gini'): np.float64(0.850625), (3, 'entropy'): np.float64(0.825), (5, 'gini'): np.float64(0.945625), (5, 'entropy'): np.float64(0.93625), (10, 'gini'): np.float64(0.9956250000000001), (10, 'entropy'): np.float64(1.0), (None, 'gini'): np.float64(1.0), (None, 'entropy'): np.float64(1.0)}
    Altura Criterio  Accuracy Train  Accuracy Validación
0      1.0  entropy

Claramente a medida de que aumentamos el max_depth del arbol, vemos que la accuracy en los datos de train tiende a 100%, lo que tiene sentido, ya que calsifica bien todos estos datos (los memoriza). A su vez, para los datos de test. resulta peor. observamos overfitting.

Estuvimos probando con otros valores y llegamos a la conclusion que la mejor altura es 2, ya que no solo, comparado a la performarnce en el test con otra alturas, es mejor, sino que tambien la diferencia con los datos de training es menor, osea que esta generalizando bastante bien, y no memoriza solo los datos de entrenamiento.


Ejercicio 3
Importante: de acá en más sólamente utilizaremos el score promedio cuando hagamos K-fold cross-validation.

Para el arbol de decision elegimos jugar con los siguientes hiperparametros:


*   Max_depth (para regular el tamaño del arbol)
*   class_weight (para tratar el desbalanceo de clases que tenemos)
*   criterion
*   min_samples_split (para definir cuantos hijos vamos a tener por rama como maximo)

Por otro lado, para knn elegimos:
* La cantidad de vecinos (k)
* weights (darle mas peso a los que estan mas cercanos y menos a los mas alejados)
* metrics (el tipo de distancia que usamos)

Finalmente para SVM:
* C (penalizacion por clasificacion equivocada)
* kernel (hay distintos tipos)
* class_weight (al ser desbalanceada nos conviene jugar con esto)
